In [ ]:
#!pip install nba_api
#!pip install azure-identity azure-storage-file-datalake
#import urllib.request
#url = "https://raw.githubusercontent.com/microsoft/fabric-toolbox/main/tools/OpenMirroringPythonSDK/openmirroring_operations.py"
#urllib.request.urlretrieve(url, "openmirroring_operations.py")
#!pip install pyarrow

In [ ]:
from nba_api.stats.endpoints import playergamelog
import pandas as pd
from openmirroring_operations import OpenMirroringClient
from datetime import datetime
import os

In [3]:
# Example: Get LeBron James's game logs for 2024–25
lebron_id = 2544  # Replace with desired player ID
gamelog = playergamelog.PlayerGameLog(player_id=lebron_id, season='2024-25', season_type_all_star='Regular Season')
df = gamelog.get_data_frames()[0]
df['Timestamp'] = datetime.now().isoformat()

In [ ]:
# Replace with your actual values
client_id = os.environ["FABRIC_CLIENT_ID"]
tenant_id = os.environ["FABRIC_TENANT_ID"]
client_secret = os.environ["FABRIC_CLIENT_SECRET"]


# Replace with your actual OneLake landing zone URL
host = "https://onelake.dfs.fabric.microsoft.com/66503be7-cb73-4bf0-b14c-dfcdd90f8f13/54f92d2b-e886-4521-9e7a-b339c1917d8f/Files/LandingZone"

# Initialize the client
client = OpenMirroringClient(
    client_id=client_id,
    client_secret=client_secret,
    client_tenant=tenant_id,
    host=host
)

In [5]:
#status = client.get_mirrored_database_status()
#print("Mirrored DB Status:", status)

In [6]:
client.create_table(
    schema_name="nba_stats",
    table_name="player_game_logs",
    key_cols=["Player_ID", "Game_ID"]
)

Folder and _metadata.json created successfully at: nba_stats.schema/player_game_logs


In [7]:
# Save as Parquet
df.to_parquet("lebron_game_logs.parquet", index=False)

# Upload to Fabric
client.upload_data_file(
    schema_name="nba_stats",
    table_name="player_game_logs",
    local_file_path="lebron_game_logs.parquet"
)

File uploaded successfully as '_00000000000000000001.parquet'.
File renamed from _00000000000000000001.parquet to 00000000000000000001.parquet successfully.
File renamed successfully to '00000000000000000001.parquet'.
